In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import plotly.express as px
import seaborn as sns

# Load dataset
df = pd.read_csv('train.csv')  # Replace with your actual file path


In [223]:
# Select only numerical columns with NaN values
numerical_missing = df.select_dtypes(include='number').columns[df.select_dtypes(include='number').isnull().any()]

print(numerical_missing)

Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')


In [224]:
# Select only object or category columns
cat_cols = df.select_dtypes(include=["object", "category"]).columns

# Fill NaN values in only those columns
df[cat_cols] = df[cat_cols].fillna("Missing")

## 1.1 convert types

In [225]:
#from Id	LotArea	LotShape	BldgType	HouseStyle	BsmtFinSF1	BsmtFinType2	BsmtFinSF2	BsmtUnfSF	TotalBsmtSF	1stFlrSF	2ndFlrSF	
#GrLivArea	BsmtFullBath	BsmtHalfBath	FullBath	HalfBath	BedroomAbvGr	KitchenAbvGr	TotRmsAbvGrd	GarageCars	GarageArea
#we drop

df[['MSSubClass', 'MoSold', 'YearBuilt', 'YearRemodAdd', 'YrSold']] = df[['MSSubClass', 'MoSold', 'YearBuilt', 'YearRemodAdd', 'YrSold']].astype('object')

In [226]:
threshold = 0.30  # 30%

# Get missing value ratio per column
missing_ratio = df.isna().mean()

# Identify columns to drop
missing_ratio[missing_ratio > threshold].index.tolist()

[]

Keeping features with more than 30% missing values can still improve your model score if those features:

1. Have Strong Predictive Power
Even if many values are missing, the non-missing values might strongly correlate with house prices. For example:

A rare feature like "Luxury Finish Quality" might be present in only 40% of houses but highly predictive of high prices.

2. Missingness Is Informative
The fact that a value is missing can itself be predictive. For instance:

"Pool Size" missing might imply no pool → possibly lower price.

So, missing = meaningful, not just a problem.

3. Effective Imputation
If you handle missing values well (e.g., with mean/median, KNN, or even a separate "missing" category), the feature can still contribute positively without hurting the model.

4. Regularized Models Can Handle Noise
Algorithms like XGBoost, LightGBM, and Random Forest can handle missing values internally or tolerate noisy features due to their robustness.

🔑 In Short:
Dropping features just based on missingness may cause you to lose valuable information. If the signal outweighs the noise, even sparse features can boost prediction accuracy.

## 1.4 keep means for missing quantitative + use them train

In [227]:
#save numeric means for missing test
means = df.select_dtypes(include='number').mean()

#imputer numeric par moyenne dans train
df = df.fillna(0)

C:\Users\berra\AppData\Local\Temp\ipykernel_4132\3383472244.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)


## 1.5 create variables

In [228]:
df_dropped = df.copy()

#'YearBuilt', 'YearRemodAdd, 'MoSold', 'YrSold' convert to quantitative
# List of columns to convert
columns_to_convert = ['YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold']

# Convert each to integer safely
for col in columns_to_convert:
    df_dropped.loc[:, col] = df_dropped[col].astype(int)

# Set reference year
reference_year = df_dropped['YrSold'].max()

# Create quantitative features safely
df_dropped.loc[:, 'HouseAge'] = reference_year - df_dropped['YearBuilt']
df_dropped.loc[:, 'YearsSinceRemod'] = reference_year -df_dropped['YearRemodAdd']
df_dropped.loc[:, 'TimeIndex'] = (
    (df_dropped['YrSold'] - df_dropped['YrSold'].min()) * 12 + df_dropped['MoSold']
)

df_dropped = df_dropped.drop([
    'YearBuilt', 'YearRemodAdd', 'YrSold', 'MoSold'
], axis=1) 

# Convert object columns to numeric, forcing errors to NaN
cols_to_convert = ['HouseAge', 'YearsSinceRemod', 'TimeIndex']
df_dropped[cols_to_convert] = df_dropped[cols_to_convert].apply(pd.to_numeric, errors='coerce')

## 1.6 correct skewness

In [229]:
#case 1: only binary no log

#Most houses don't have low-quality finished square footage.
#Very few do, with highly varied amounts.
cols_to_binary_only_0 = ['BsmtHalfBath', 'EnclosedPorch', 'ScreenPorch']

for col in cols_to_binary_only_0:
    df_dropped[f'Has{col}'] = (df_dropped[col] > 0).astype(int)
    df_dropped.drop(columns=[col], inplace=True)

In [230]:
    

# = 1 or not

df_dropped['HasKitchen'] = (df_dropped['KitchenAbvGr'] == 1).astype(int)
df_dropped.drop(columns=['KitchenAbvGr'], inplace=True)


#case 2: binary+log

cols = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '2ndFlrSF', 'WoodDeckSF', 'OpenPorchSF']  # Replace with actual column names

for col in cols:
    df_dropped[f'Has{col}'] = (df_dropped[col] > 0).astype(int)
    df_dropped[f'{col}_log'] = np.log1p(df_dropped[col])

In [231]:
for col in cols:
    non_numeric = df_dropped[col].apply(lambda x: not pd.api.types.is_number(x))
    if non_numeric.any():
        print(f"Column '{col}' contains non-numeric values:")
        print(df_dropped.loc[non_numeric, col].unique())

In [232]:
#case 3: just drop 
df_dropped.drop(columns=['LowQualFinSF','3SsnPorch', 'PoolArea', 'MiscVal'], inplace=True)

# case 4: log only

#correlation = df_cat_quant['HouseAge'].corr(df_cat_quant['YearsSinceRemod'], method='pearson')
#print(f"Pearson correlation: {correlation:.3f}")
df_dropped.drop(columns=[ 'TimeIndex'], inplace=True)# cyclique, pas significative

# List of variables to transform
vars_to_log = ['LotFrontage', 'LotArea', 'TotalBsmtSF', 'GrLivArea',  'SalePrice', 'BsmtUnfSF', '1stFlrSF']

# Create log-transformed versions with "_log" suffix
for col in vars_to_log:
    df_dropped[col + '_log'] = np.log1p(df_dropped[col])  # log1p handles zero safely

# Drop the original columns
df_dropped.drop(columns=vars_to_log, inplace=True)

### cap

## 1.7 cap outliers

In [233]:
# cap outliers
#outliers cap

num_features = [col for col in df_dropped.select_dtypes(include='number') if col != 'Id']
 
# Store the limits for each column
caps = {}

for col in num_features:
    q_low = df_dropped[col].quantile(0.01)
    q_high = df_dropped[col].quantile(0.99)
    
    # Save the thresholds
    caps[col] = (q_low, q_high)
    
    # Apply clipping
    df_dropped[col] = df_dropped[col].clip(lower=q_low, upper=q_high)



## 1.8 Categorical variable (missing+rare categories)

In [234]:
# One-hot encode all object or category dtype columns
df_dropped = pd.get_dummies(df_dropped)

## 1.9 predict

In [235]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

# Step 1: Separate features and target
X_train = df_dropped.drop(columns=['Id', 'SalePrice_log'])  # Replace 'Price' with your actual target if named differently
y = df_dropped['SalePrice_log']

model = GradientBoostingRegressor(n_estimators=1100, loss='squared_error', subsample = 0.35, learning_rate = 0.05,random_state=1)

model.fit(X_train, y)


GradientBoostingRegressor(learning_rate=0.05, n_estimators=1100, random_state=1,
                          subsample=0.35)

# Test Quantitative columns (outliers capping...)

In [236]:
# test

#from Id	LotArea	LotShape	BldgType	HouseStyle	BsmtFinSF1	BsmtFinType2	BsmtFinSF2	BsmtUnfSF	TotalBsmtSF	1stFlrSF	2ndFlrSF	
#GrLivArea	BsmtFullBath	BsmtHalfBath	FullBath	HalfBath	BedroomAbvGr	KitchenAbvGr	TotRmsAbvGrd	GarageCars	GarageArea
#we drop
d_test = pd.read_csv('test.csv') 

In [237]:
# Select only numerical columns with NaN values
numerical_missing = d_test.select_dtypes(include='number').columns[d_test.select_dtypes(include='number').isnull().any()]

print(numerical_missing)

Index(['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
       'GarageCars', 'GarageArea'],
      dtype='object')


## 2.1 convert types

In [238]:
d_test[['MSSubClass', 'MoSold', 'YearBuilt', 'YearRemodAdd', 'YrSold']] = d_test[['MSSubClass', 'MoSold', 'YearBuilt', 'YearRemodAdd', 'YrSold']].astype('object')

## 2.2 drop most missing

In [239]:
# Select only object or category columns
cat_cols = d_test.select_dtypes(include=["object", "category"]).columns

# Fill NaN values in only those columns
d_test[cat_cols] = d_test[cat_cols].fillna("Missing")

C:\Users\berra\AppData\Local\Temp\ipykernel_4132\2363930934.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  d_test[cat_cols] = d_test[cat_cols].fillna("Missing")


## 2.4 fill na with means of train

In [240]:
# Imputation dans le test set (numeric)
d_test.fillna(0, inplace=True)

## 2.5 create variables

In [241]:
#'YearBuilt', 'YearRemodAdd, 'MoSold', 'YrSold' convert to quantitative
# List of columns to convert
columns_to_convert = ['YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold']

# Convert each to integer safely
for col in columns_to_convert:
    d_test.loc[:, col] = d_test[col].astype(int)

# Set reference year
reference_year = d_test['YrSold'].max()

# Create quantitative features safely
d_test.loc[:, 'HouseAge'] = reference_year - d_test['YearBuilt']
d_test.loc[:, 'YearsSinceRemod'] = reference_year - d_test['YearRemodAdd']


d_test = d_test.drop([
    'YearBuilt', 'YearRemodAdd', 'YrSold', 'MoSold'
], axis=1) 

# Convert object columns to numeric, forcing errors to NaN
cols_to_convert = ['HouseAge', 'YearsSinceRemod']
d_test[cols_to_convert] = d_test[cols_to_convert].apply(pd.to_numeric, errors='coerce')

## 2.6 correct skewness

In [242]:
#case 1: only binary no log

#Most houses don't have low-quality finished square footage.
#Very few do, with highly varied amounts.
cols_to_binary_only_0 = ['BsmtHalfBath', 'EnclosedPorch', 'ScreenPorch']

for col in cols_to_binary_only_0:
    d_test[f'Has{col}'] = (d_test[col] > 0).astype(int)
    d_test.drop(columns=[col], inplace=True)

# = 1 or not

d_test['HasKitchen'] = (d_test['KitchenAbvGr'] == 1).astype(int)
d_test.drop(columns=['KitchenAbvGr'], inplace=True)


#case 2: binary+log

cols = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '2ndFlrSF', 'WoodDeckSF', 'OpenPorchSF']  # Replace with actual column names

for col in cols:
    d_test[f'Has{col}'] = (d_test[col] > 0).astype(int)
    d_test[f'{col}_log'] = np.log1p(d_test[col])

#case 3: just drop 
d_test.drop(columns=['LowQualFinSF','3SsnPorch', 'PoolArea', 'MiscVal'], inplace=True)

# case 4: log only

#correlation = df_cat_quant['HouseAge'].corr(df_cat_quant['YearsSinceRemod'], method='pearson')
#print(f"Pearson correlation: {correlation:.3f}")

# List of variables to transform
vars_to_log = ['LotFrontage', 'LotArea', 'TotalBsmtSF', 'GrLivArea',  'BsmtUnfSF', '1stFlrSF']


# Create log-transformed versions with "_log" suffix
for col in vars_to_log:
    d_test[col + '_log'] = np.log1p(d_test[col])  # log1p handles zero safely

# Drop the original columns
d_test.drop(columns=vars_to_log, inplace=True)

## 2.7 cap outliers

In [243]:
num_features = [col for col in num_features if col != "SalePrice_log"]

for col in num_features:
    q_low, q_high = caps[col]
    d_test[col] = d_test[col].clip(lower=q_low, upper=q_high)


## 2.8 Categorical variable (missing+rare categories)

In [244]:
# One-hot encode test set
X_test = pd.get_dummies(d_test)
X_test = d_test.drop(columns=['Id'], errors='ignore') 
# Align with training columns (very important!)
X_test_aligned = X_test.reindex(columns=X_train.columns, fill_value=0)


y_pred_log = model.predict(X_test_aligned)
y_pred = np.expm1(y_pred_log)  # reverse np.log1p()
d_test['SalePrice'] = y_pred
d_test[['Id', 'SalePrice']].to_csv("predictions_g.csv", index=False)

In [245]:

pd.set_option('display.max_columns', None)
print(X_test_aligned.columns)

Index(['MSSubClass', 'OverallQual', 'OverallCond', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', '2ndFlrSF', 'BsmtFullBath', 'FullBath', 'HalfBath',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=310)


In [246]:
print(list(df_dropped.columns)) 

['Id', 'MSSubClass', 'OverallQual', 'OverallCond', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', '2ndFlrSF', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'HouseAge', 'YearsSinceRemod', 'HasBsmtHalfBath', 'HasEnclosedPorch', 'HasScreenPorch', 'HasKitchen', 'HasMasVnrArea', 'MasVnrArea_log', 'HasBsmtFinSF1', 'BsmtFinSF1_log', 'HasBsmtFinSF2', 'BsmtFinSF2_log', 'HasBsmtUnfSF', 'BsmtUnfSF_log', 'Has2ndFlrSF', '2ndFlrSF_log', 'HasWoodDeckSF', 'WoodDeckSF_log', 'HasOpenPorchSF', 'OpenPorchSF_log', 'LotFrontage_log', 'LotArea_log', 'TotalBsmtSF_log', 'GrLivArea_log', 'SalePrice_log', '1stFlrSF_log', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl', 'Alley_Missing', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lv

In [247]:
import os
os.getcwd()

'C:\\Users\\berra\\House Prices'